In [104]:
import numpy as np
import string

from qiskit.circuit.library import TwoLocal
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp

from qiskit_addon_cutting import partition_problem
from qiskit_addon_cutting import generate_cutting_experiments

from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2, Batch

In [118]:
# constructing quantum circuit of same structure as given in the qasm file

num_qubits = 20
reps = 4
gate_sequence = ['rz','sx','rz','sx','x','rz']

# nearest neighbour configuration
seq_pairs = [(i,(i+1)%num_qubits) for i in range(0,num_qubits,2)]
warp_pairs = [(i,(i-1)%num_qubits) for i in range(0,num_qubits,2)]

# qc two local
qc = TwoLocal(num_qubits,rotation_blocks=['rz','sx','rz','sx','x','rz'],entanglement_blocks='cz', entanglement=[seq_pairs,warp_pairs], reps=reps,skip_final_rotation_layer=True).decompose()  
qc.assign_parameters([np.random.random(1)[0] for i in range(len(qc.parameters))], inplace=True)
#qc.measure_all()
qc.draw(scale=0.8)

┌─────────────┐ ┌────┐  ┌────────────┐ ┌────┐┌───┐┌─────────────┐    »
 q_0: ─┤ Rz(0.26396) ├─┤ √X ├──┤ Rz(0.7345) ├─┤ √X ├┤ X ├┤ Rz(0.55978) ├──■─»
       ├─────────────┤ ├────┤ ┌┴────────────┤ ├────┤├───┤├─────────────┤  │ »
 q_1: ─┤ Rz(0.75997) ├─┤ √X ├─┤ Rz(0.30523) ├─┤ √X ├┤ X ├┤ Rz(0.16496) ├──■─»
       ├─────────────┤ ├────┤ ├─────────────┤ ├────┤├───┤├─────────────┤    »
 q_2: ─┤ Rz(0.22893) ├─┤ √X ├─┤ Rz(0.40285) ├─┤ √X ├┤ X ├┤ Rz(0.73524) ├──■─»
       ├─────────────┤ ├────┤┌┴─────────────┴┐├────┤├───┤├─────────────┤  │ »
 q_3: ─┤ Rz(0.45499) ├─┤ √X ├┤ Rz(0.0042912) ├┤ √X ├┤ X ├┤ Rz(0.42727) ├──■─»
       ├─────────────┤ ├────┤└┬─────────────┬┘├────┤├───┤├─────────────┤    »
 q_4: ─┤ Rz(0.24675) ├─┤ √X ├─┤ Rz(0.40363) ├─┤ √X ├┤ X ├┤ Rz(0.78481) ├──■─»
       ├─────────────┤ ├────┤ ├─────────────┤ ├────┤├───┤├─────────────┤  │ »
 q_5: ─┤ Rz(0.49666) ├─┤ √X ├─┤ Rz(0.84078) ├─┤ √X ├┤ X ├┤ Rz(0.25754) ├──■─»
       ├─────────────┤ ├────┤ ├─────────────┤ ├────┤├───┤├─────────────┤    »
 q_6: ─┤ Rz(0.63751) ├─┤ √X ├─┤ Rz(0.99867) ├─┤ √X ├┤ X ├┤ Rz(0.79911) ├──■─»
       ├─────────────┤ ├────┤ ├─────────────┤ ├────┤├───┤├─────────────┤  │ »
 q_7: ─┤ Rz(0.59969) ├─┤ √X ├─┤ Rz(0.52808) ├─┤ √X ├┤ X ├┤ Rz(0.68695) ├──■─»
       ├─────────────┤ ├────┤ ├─────────────┤ ├────┤├───┤├─────────────┤    »
 q_8: ─┤ Rz(0.10184) ├─┤ √X ├─┤ Rz(0.43671) ├─┤ √X ├┤ X ├┤ Rz(0.57901) ├──■─»
       ├─────────────┤ ├────┤ ├─────────────┤ ├────┤├───┤├─────────────┤  │ »
 q_9: ─┤ Rz(0.10175) ├─┤ √X ├─┤ Rz(0.53216) ├─┤ √X ├┤ X ├┤ Rz(0.70243) ├──■─»
       ├─────────────┤ ├────┤ ├─────────────┤ ├────┤├───┤├─────────────┤    »
q_10: ─┤ Rz(0.53442) ├─┤ √X ├─┤ Rz(0.79622) ├─┤ √X ├┤ X ├┤ Rz(0.60137) ├──■─»
       ├─────────────┤ ├────┤ ├─────────────┤ ├────┤├───┤├─────────────┤  │ »
q_11: ─┤ Rz(0.36369) ├─┤ √X ├─┤ Rz(0.10784) ├─┤ √X ├┤ X ├┤ Rz(0.42739) ├──■─»
      ┌┴─────────────┴┐├────┤ ├─────────────┴┐├────┤├───┤├─────────────┤    »
q_12: ┤ Rz(0.0030291) ├┤ √X ├─┤ Rz(0.092739) ├┤ √X ├┤ X ├┤ Rz(0.40615) ├──■─»
      └┬─────────────┬┘├────┤┌┴──────────────┤├────┤├───┤├─────────────┤  │ »
q_13: ─┤ Rz(0.57464) ├─┤ √X ├┤ Rz(0.0055998) ├┤ √X ├┤ X ├┤ Rz(0.29392) ├──■─»
       ├─────────────┤ ├────┤└┬─────────────┬┘├────┤├───┤├─────────────┤    »
q_14: ─┤ Rz(0.61177) ├─┤ √X ├─┤ Rz(0.81356) ├─┤ √X ├┤ X ├┤ Rz(0.47852) ├──■─»
       ├─────────────┤ ├────┤ ├─────────────┤ ├────┤├───┤├─────────────┤  │ »
q_15: ─┤ Rz(0.72469) ├─┤ √X ├─┤ Rz(0.63042) ├─┤ √X ├┤ X ├┤ Rz(0.75789) ├──■─»
       ├─────────────┤ ├────┤ ├─────────────┤ ├────┤├───┤├─────────────┤    »
q_16: ─┤ Rz(0.67801) ├─┤ √X ├─┤ Rz(0.45413) ├─┤ √X ├┤ X ├┤ Rz(0.84461) ├──■─»
       ├─────────────┤ ├────┤ ├─────────────┤ ├────┤├───┤├─────────────┤  │ »
q_17: ─┤ Rz(0.43937) ├─┤ √X ├─┤ Rz(0.99982) ├─┤ √X ├┤ X ├┤ Rz(0.84215) ├──■─»
       ├─────────────┤ ├────┤ ├─────────────┤ ├────┤├───┤├─────────────┤    »
q_18: ─┤ Rz(0.56909) ├─┤ √X ├─┤ Rz(0.64899) ├─┤ √X ├┤ X ├┤ Rz(0.28588) ├──■─»
       ├─────────────┴┐├────┤ ├─────────────┤ ├────┤├───┤├─────────────┴┐ │ »
q_19: ─┤ Rz(0.090684) ├┤ √X ├─┤ Rz(0.70971) ├─┤ √X ├┤ X ├┤ Rz(0.033898) ├─■─»
       └──────────────┘└────┘ └─────────────┘ └────┘└───┘└──────────────┘   »
«       ┌────────────┐ ┌────┐┌─────────────┐ ┌────┐┌───┐ ┌────────────┐      »
« q_0: ─┤ Rz(0.2581) ├─┤ √X ├┤ Rz(0.84157) ├─┤ √X ├┤ X ├─┤ Rz(0.1516) ├────■─»
«       ├────────────┤ ├────┤├─────────────┤ ├────┤├───┤┌┴────────────┤    │ »
« q_1: ─┤ Rz(0.8142) ├─┤ √X ├┤ Rz(0.83618) ├─┤ √X ├┤ X ├┤ Rz(0.79707) ├─■──┼─»
«      ┌┴────────────┤ ├────┤├─────────────┤ ├────┤├───┤└┬────────────┤ │  │ »
« q_2: ┤ Rz(0.12759) ├─┤ √X ├┤ Rz(0.18602) ├─┤ √X ├┤ X ├─┤ Rz(0.3744) ├─■──┼─»
«      ├─────────────┤ ├────┤├─────────────┤ ├────┤├───┤┌┴────────────┤    │ »
« q_3: ┤ Rz(0.60057) ├─┤ √X ├┤ Rz(0.62174) ├─┤ √X ├┤ X ├┤ Rz(0.71958) ├─■──┼─»
«      ├─────────────┴┐├────┤├─────────────┤ ├────┤├───┤├─────────────┤ │  │ »
« q_4: ┤ Rz(0.020192) ├┤ √X ├┤ Rz(0.67026) ├─┤ √X ├┤ X ├┤ Rz(0.77964) ├─■──┼─»
«      ├───────────

In [50]:
num_qubits = qc.num_qubits
num_qubits

10

In [51]:
def cutting_circuits(qc,num_cuts):

    num_qubits = qc.num_qubits
    observable = SparsePauliOp(["Z"*num_qubits])

    labels  = list(string.ascii_uppercase[:num_cuts])
    partition_string = ''.join([label*(num_qubits//num_cuts) for label in labels])

    partitioned_problem = partition_problem(
    circuit=qc, partition_labels=partition_string,observables=observable.paulis)

    subcircuits = partitioned_problem.subcircuits
    subobservables = partitioned_problem.subobservables
    bases = partitioned_problem.bases

    return(subcircuits,subobservables,bases)

In [52]:
subcircuits,subobservables,bases = cutting_circuits(qc,5)

In [53]:
num_samples = 10

subexperiments, coefficients = generate_cutting_experiments(
    circuits=subcircuits, observables=subobservables, num_samples=num_samples
)

In [54]:
backend = AerSimulator()

pass_manager = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_subexperiments = {
    label: pass_manager.run(partition_subexpts)
    for label, partition_subexpts in subexperiments.items()
}

with Batch(backend=backend) as batch:
    sampler = SamplerV2(mode=batch)
    jobs = {
        label: sampler.run(subsystem_subexpts, shots=2**12).result()
        for label, subsystem_subexpts in isa_subexperiments.items()
    }

In [55]:
def knitting_results(job_dict): 

    final_state = ''

    for results in job_dict.values():
        trials = 0
        final_counts = {}
        for result in results:
            counts = result.data.observable_measurements.get_counts()
            trials += sum(list(counts.values()))
            for keys in counts.keys():
                final_counts[keys] = final_counts.get(keys, 0)+counts[keys]
        
        final_state+=max(final_counts,key=final_counts.get)
    
    return(final_state)

In [56]:
knitting_results(jobs)

'0000000000'

In [57]:
## Testing the output of original circuit
qc.measure_all()
sampler = SamplerV2(mode=backend)
result = sampler.run([qc],shots=2**12).result()

In [58]:
counts = result[0].data.meas.get_counts()
max(counts,key=counts.get)

'0000000000'

In [59]:
counts

{'0101000000': 42,
 '1011000000': 16,
 '0010001000': 39,
 '0000000000': 334,
 '0000010010': 41,
 '1101000000': 14,
 '0001000100': 23,
 '0100000000': 72,
 '0100001000': 30,
 '1000001100': 13,
 '0001001001': 6,
 '0000011000': 59,
 '0000001000': 138,
 '0010011010': 5,
 '1000000010': 41,
 '0000010000': 142,
 '0000000010': 93,
 '1000001010': 22,
 '0000100000': 49,
 '0101100010': 2,
 '0000000100': 53,
 '0011011000': 11,
 '0001010000': 77,
 '0001100000': 21,
 '1001000000': 54,
 '0001000010': 55,
 '1000011000': 35,
 '0001000000': 171,
 '1011010000': 11,
 '0100010001': 5,
 '0001001000': 75,
 '0010000010': 29,
 '1010010100': 3,
 '0001011000': 28,
 '0010001010': 11,
 '0000010110': 4,
 '0011000000': 42,
 '1110100100': 1,
 '0001001010': 23,
 '0100011000': 18,
 '1000000000': 142,
 '0001010010': 31,
 '0101011100': 1,
 '1000000100': 25,
 '0100011010': 5,
 '1000010000': 59,
 '0000000011': 2,
 '1010011000': 12,
 '1000010110': 5,
 '1000001000': 64,
 '0000011100': 9,
 '0100101000': 5,
 '1000101010': 2,
 '

## Parallelising Runs

In [119]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

In [120]:
# Generate subcircuits and observables
subcircuits, subobservables, bases = cutting_circuits(qc, 2)
num_samples = 10

subexperiments, coefficients = generate_cutting_experiments(
    circuits=subcircuits, observables=subobservables, num_samples=num_samples
)

In [121]:
# Backend and pass manager setup
backend = AerSimulator()
pass_manager = generate_preset_pass_manager(optimization_level=1, backend=backend)

# Prepare subexperiments with pass manager
isa_subexperiments = {
    label: pass_manager.run(partition_subexpts)
    for label, partition_subexpts in subexperiments.items()
}

In [122]:
# Function to run a batch of subexperiments on a thread
def run_subexperiment(label, sub_experiment):
    with Batch(backend=backend) as batch:
        sampler = SamplerV2(mode=batch)
        job = sampler.run(sub_experiment, shots=2**12).result()
        return label, job


In [123]:
# Using ThreadPoolExecutor for parallel processing

start_time = time.time()
job_results = {}
with ThreadPoolExecutor() as executor:
    future_jobs = {executor.submit(run_subexperiment, label, sub_experiment): label
                   for label, sub_experiment in isa_subexperiments.items()}

    for future in as_completed(future_jobs):
        label = future_jobs[future]
        try:
            result_label, job_result = future.result()
            job_results[result_label] = job_result
        except Exception as exc:
            print(f"Experiment {label} generated an exception: {exc}")
            
end_time = time.time()

In [124]:
# Print the final state with maximum probability
print('State of Maximum Probability:', knitting_results(job_results))
execution_time = end_time - start_time
print(f'Total Execution Time: {execution_time:.2f} seconds')

State of Maximum Probability: 00000000001000000000
Total Execution Time: 10.18 seconds
